# Benchmark of waveform upload to Tektronix AWG70002A

This notebook investigates the time it takes to send waveforms from the PC (from QCoDeS) to the Tektronix awg70002A.

We generate 10 sine waves of each 100,000 points and make a sequence of each of them being played twice before proceeding to the next in line. We upload the waveforms as 8-bit signals along with two markers.

In [1]:
import numpy as np
import time
from qcodes.instrument_drivers.tektronix.AWG70002A import AWG70002A

In [2]:
awg = AWG70002A('awg', 'TCPIP0::172.20.2.101::inst0::INSTR')

Connected to: TEKTRONIX AWG70002A (serial:B020397, firmware:FV:5.3.0128.0) in 0.32s


## Building the waveforms

This should be done with broadbean but due to a time constraint, we simply use numpy for now.

In [174]:
N = 100000  # points in each waveform
M = 10  # no. of waveforms

twopi = np.linspace(0, 2*np.pi, N)
wfms = []
m1s = []
m2s = []

m1 = np.concatenate((np.zeros(int(N/2)), np.ones(int(N/2))))
m2 = np.concatenate((np.ones(int(N/2)), np.zeros(int(N/2))))

for m in range(M):
    wfms.append(np.sin((m+1)*0.25*twopi))
    m1s.append(m1)
    m2s.append(m2)
    

## Building a .seqx file

In [175]:
# First we must can waveforms and markers as binary .wfmx files...

wfmx_files = []
for m in range(M):
    data = np.array([wfms[m], m1s[m], m2s[m]])
    wfmx = awg.makeWFMXFile(data)
    wfmx_files.append(wfmx)

In [176]:
# ...next we should provide sequencing information...

trig_waits = [2]*M
nreps = [2]*M
event_jumps = [0]*M
event_jump_to = [0]*M
go_to = [0]*M

In [177]:
np.shape(wfmx_files)
type(wfmx_files)

list

In [178]:
awg.sendWFMXFile(wfmx_files[0], 'benchmark.wfmx')
awg.loadWFMXFile('benchmark.wfmx')

In [179]:
# ...and then we may build the .seqx file.
# We put the waveforms on one channel

seqx_file = awg.makeSEQXFile(trig_waits,
                             nreps,
                             event_jumps,
                             event_jump_to,
                             go_to,
                             [wfmx_files, wfmx_files])

In [180]:
# Now comes the part to benchmark, which we put in a function

def benchmark_seqx(seqx_file, no_of_reps=10):
    times = np.zeros(no_of_reps)
    
    # bootstrap
    awg.clearSequenceList()
    awg.clearWaveformList()
    
    for n in range(no_of_reps):
        t_start = time.perf_counter()
        
        awg.sendSEQXFile(seqx_file, 'benchmarkingqcodes.seqx')
        awg.ask("*OPC?")
        awg.loadSEQXFile('benchmarkingqcodes.seqx')
        awg.ch1.setSequenceTrack('Sequence', 1)
        
        t_stop = time.perf_counter()
        
        times[n] = t_stop - t_start
        
        print('Completed run {} of {}.'.format(n+1, no_of_reps))
        
    return times

In [182]:
times = benchmark_seqx(seqx_file, no_of_reps=10)
print('-'*25)
print('Benchmark complete.')
print('Time spent: {} s, deviation {} s'.format(np.mean(times), np.std(times)))

-------------------------
Benchmark complete.
Time spent: 6.728395942400129 s, deviation 0.36213886283086644 s


In [9]:
awg.close()